<a href="https://colab.research.google.com/github/Sarvesh1814/US-Book-Recommendation-System/blob/main/Content_based_BOOK_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
import warnings
warnings.filterwarnings('ignore')

In [117]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

### Mount drive and load datasets

In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
Base_path ='/content/drive/MyDrive/Data/RS-3'

#### Books with ISBN

In [120]:
Book_ISBN = pd.read_csv(Base_path+"/Books.csv")

In [121]:
Book_ISBN.rename(columns = {'Book-Title':'Book_Title'}, inplace = True)

#### Books with Description

In [122]:
Books = pd.read_csv(Base_path+"/Books_Final.csv")

In [123]:
Books.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description
0,0,0,Footfall,en,706,['Fiction'],"Hostile extraterrestrials, known as the Fithp,..."
1,1,1,The Reader,en,224,['Fiction'],Hailed for its coiled eroticism and the moral ...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],Eleven-year-old Nolitye's granny used to say: ...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],This New York Times bestseller of a troubled f...
4,4,4,The Hours: A Novel,en,295,['Fiction'],Michael Cunningham brings together his Pulitze...


In [124]:
df=Books

### Rating Data

In [125]:
Ratings = pd.read_csv(Base_path+"/Ratings.csv")

In [126]:
Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### NLP preprocessing on Description Text

In [127]:
import nltk
nltk.download('omw-1.4')
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [128]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [129]:
def clean_text(text):
    """
    Clean the text by removing punctuation, stopwords, and converting to lowercase.
    """
    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    
    # Convert to lowercase
    text = text.lower()

    # Remove non-English characters
    text = re.sub(r"[^\x00-\x7F]+", "", text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the words back into a single string
    cleaned_text = " ".join(words)
    
    return cleaned_text

In [130]:
df["category"] = df["categories"].apply(clean_text)

In [131]:
df["category"].head()

0    fiction
1    fiction
2    fiction
3    fiction
4    fiction
Name: category, dtype: object

In [132]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category
0,0,0,Footfall,en,706,['Fiction'],"Hostile extraterrestrials, known as the Fithp,...",fiction
1,1,1,The Reader,en,224,['Fiction'],Hailed for its coiled eroticism and the moral ...,fiction
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],Eleven-year-old Nolitye's granny used to say: ...,fiction
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],This New York Times bestseller of a troubled f...,fiction
4,4,4,The Hours: A Novel,en,295,['Fiction'],Michael Cunningham brings together his Pulitze...,fiction


In [133]:
# Replace all non-letters with spaces # Search for all non-letters
# define the function to remove non-alphabetic characters
def remove_non_alphabetic(text):
    return re.sub("[^a-zA-Z]", " ", str(text))

# apply the function to each row of the "description" column
df["description"] = df["description"].apply(remove_non_alphabetic)

In [134]:
df["description"] = df["description"].apply(lambda x: clean_text(x))

### Cleaning the Categories Column

In [135]:
import string

# Create a translation table for punctuation removal
translator = str.maketrans(' ', ' ', string.punctuation)

# Apply the translation table to the 'Review' column of the data frame
df['category'] = df['category'].str.translate(translator)

In [136]:
df.drop('categories', axis='columns')

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,description,category
0,0,0,Footfall,en,706,hostile extraterrestrial known fithp invade ea...,fiction
1,1,1,The Reader,en,224,hailed coiled eroticism moral claim make upon ...,fiction
2,2,2,Hidden Star (The Star Series),en,234,eleven year old nolitye granny used say mess w...,fiction
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,new york time bestseller troubled family vermo...,fiction
4,4,4,The Hours: A Novel,en,295,michael cunningham brings together pulitzer pr...,fiction
...,...,...,...,...,...,...,...
6570,6570,6570,Art of Peace (Shambhala Pocket Classics),en,192.0,inspirational teaching collection show real wa...,philosophy
6571,6571,6571,Autumn Lover,en,416.0,returning wyoming ranch civil war end elyssa s...,fiction
6572,6572,6572,Tales of Two Cities: A Persian Memoir,en,276.0,tale two city engrossing cross cultural memoir...,exile
6573,6573,6573,Heat,en,244.0,bestseller michael arroyo pitching arm throw s...,juvenile fiction


In [137]:
## Dropping Columns
# train_data = train_data.drop(columns=['rating','reviewText','categories'])

### Merging Categories and Description as major 

In [138]:
df['Features'] = df['category'].astype(str) + ' ' + df['description'].astype(str)
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...
1,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],eleven year old nolitye granny used say mess w...,fiction,fiction eleven year old nolitye granny used sa...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],new york time bestseller troubled family vermo...,fiction,fiction new york time bestseller troubled fami...
4,4,4,The Hours: A Novel,en,295,['Fiction'],michael cunningham brings together pulitzer pr...,fiction,fiction michael cunningham brings together pul...


### Review Tokenization

In [139]:
# Tokenize the data
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

features = df['Features']
tokenized_features = [word_tokenize(feature) for feature in features]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Word Embeddings

In [140]:
# Word Embeddings Creation

import gensim

# Train the Word2Vec model
model = gensim.models.Word2Vec(tokenized_features, vector_size=250, window=5, min_count=1, workers=8)
# Access the word vectors
word_vectors = model.wv

In [141]:
word_vectors

### Representing Text Review into Vectors

In [142]:
df['Features']

0       fiction hostile extraterrestrial known fithp i...
1       fiction hailed coiled eroticism moral claim ma...
2       fiction eleven year old nolitye granny used sa...
3       fiction new york time bestseller troubled fami...
4       fiction michael cunningham brings together pul...
                              ...                        
6570    philosophy inspirational teaching collection s...
6571    fiction returning wyoming ranch civil war end ...
6572    exile tale two city engrossing cross cultural ...
6573    juvenile fiction bestseller michael arroyo pit...
6574    bean fictitious character card orson scott car...
Name: Features, Length: 6575, dtype: object

In [143]:
desc_array = []
for feature in df['Features']:
  word_vector = np.zeros((1, 250))
  for word in word_tokenize(feature):
    word_vector += word_vectors[word]
  desc_array.append(word_vector) 

In [144]:
desc_array = np.array(desc_array)
desc_array = desc_array.reshape(-1,250)
desc_array.shape

(6575, 250)

In [145]:
desc_array

array([[  2.45174079,  -0.12956149,   2.33609068, ...,  -1.30430836,
         -1.57217316,   0.22415386],
       [ 21.61597282,  -0.0569641 ,  10.25233885, ...,  -8.58289987,
         -8.52895443,   2.11538759],
       [ 36.25960573,  -3.06131534,  16.59801069, ..., -12.84171893,
        -14.63442438,   0.16200922],
       ...,
       [ 23.96771626,  -4.33206995,  20.18530573, ..., -11.11567809,
        -13.78794182,   6.41919423],
       [ 42.74540315,  -1.1065219 ,  26.5575543 , ..., -15.30784593,
        -17.92274941,   1.03113462],
       [ 16.49963261,  -1.62859472,  21.74145838, ...,  -7.50330574,
        -11.25743339,   4.09908965]])

### Cosine Similarity

In [146]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(desc_array,desc_array).flatten()

In [147]:
similarities

array([1.        , 0.97671895, 0.97460112, ..., 0.98883914, 0.95873263,
       1.        ])

In [148]:
# df['Similarity'] = similarities
# df = df.sort_values("Similarity", ascending=False)


### Top-10 Recommendation

In [149]:
df.head(10) 

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...
1,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],eleven year old nolitye granny used say mess w...,fiction,fiction eleven year old nolitye granny used sa...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],new york time bestseller troubled family vermo...,fiction,fiction new york time bestseller troubled fami...
4,4,4,The Hours: A Novel,en,295,['Fiction'],michael cunningham brings together pulitzer pr...,fiction,fiction michael cunningham brings together pul...
5,5,5,Fried Green Tomatoes at the Whistle Stop Cafe,en,402,['Fiction'],folksy fresh endearing affecting fried green t...,fiction,fiction folksy fresh endearing affecting fried...
6,6,6,The Egypt Game (Yearling Newbery),en,246,['Braille books'],grade level e,braille book,braille book grade level e
7,7,7,"Mossflower (Redwall, Book 2)",en,0,['Juvenile Fiction'],soon netflix original series thrilling prequel...,juvenile fiction,juvenile fiction soon netflix original series ...
8,8,8,A Prayer for Owen Meany,en,646,['Fiction'],doomed remember boy wrecked voice voice smalle...,fiction,fiction doomed remember boy wrecked voice voic...
9,9,9,The Dilbert Principle: A Cubicle'S-Eye View of...,en,350,['Management'],nan,management,management nan


In [150]:
df.iloc[:,1]

0          0
1          1
2          2
3          3
4          4
        ... 
6570    6570
6571    6571
6572    6572
6573    6573
6574    6574
Name: Unnamed: 0, Length: 6575, dtype: int64

## TFIDF

In [151]:
df

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...
1,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...
2,2,2,Hidden Star (The Star Series),en,234,['Fiction'],eleven year old nolitye granny used say mess w...,fiction,fiction eleven year old nolitye granny used sa...
3,3,3,Songs in Ordinary Time (Oprah's Book Club (Pap...,en,682,['Fiction'],new york time bestseller troubled family vermo...,fiction,fiction new york time bestseller troubled fami...
4,4,4,The Hours: A Novel,en,295,['Fiction'],michael cunningham brings together pulitzer pr...,fiction,fiction michael cunningham brings together pul...
...,...,...,...,...,...,...,...,...,...
6570,6570,6570,Art of Peace (Shambhala Pocket Classics),en,192.0,['Philosophy'],inspirational teaching collection show real wa...,philosophy,philosophy inspirational teaching collection s...
6571,6571,6571,Autumn Lover,en,416.0,['Fiction'],returning wyoming ranch civil war end elyssa s...,fiction,fiction returning wyoming ranch civil war end ...
6572,6572,6572,Tales of Two Cities: A Persian Memoir,en,276.0,['Exiles'],tale two city engrossing cross cultural memoir...,exile,exile tale two city engrossing cross cultural ...
6573,6573,6573,Heat,en,244.0,['Juvenile Fiction'],bestseller michael arroyo pitching arm throw s...,juvenile fiction,juvenile fiction bestseller michael arroyo pit...


In [152]:
book_merged = pd.merge(Books, Book_ISBN)

In [153]:
df_rating = Ratings[Ratings['ISBN'].isin(book_merged['ISBN'])]

In [154]:
book_merged.head()

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...,0345323440,Larry Niven,1996,Del Rey Books,http://images.amazon.com/images/P/0345323440.0...,http://images.amazon.com/images/P/0345323440.0...,http://images.amazon.com/images/P/0345323440.0...
1,0,0,Footfall,en,706,['Fiction'],hostile extraterrestrial known fithp invade ea...,fiction,fiction hostile extraterrestrial known fithp i...,0345323475,Larry Niven,1985,Ballantine Books,http://images.amazon.com/images/P/0345323475.0...,http://images.amazon.com/images/P/0345323475.0...,http://images.amazon.com/images/P/0345323475.0...
2,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0679442790,Bernhard Schlink,1997,Random House,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...
3,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0375707972,Bernhard Schlink,1999,Vintage Books USA,http://images.amazon.com/images/P/0375707972.0...,http://images.amazon.com/images/P/0375707972.0...,http://images.amazon.com/images/P/0375707972.0...
4,1,1,The Reader,en,224,['Fiction'],hailed coiled eroticism moral claim make upon ...,fiction,fiction hailed coiled eroticism moral claim ma...,0679781307,Carol Brown Janeway,1998,Random House Inc,http://images.amazon.com/images/P/0679781307.0...,http://images.amazon.com/images/P/0679781307.0...,http://images.amazon.com/images/P/0679781307.0...


In [155]:
df=book_merged

In [156]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['Features'])

In [157]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [158]:
df2 = df.reset_index()
indices = pd.Series(df2.index, index=df2['ISBN'])

In [159]:
def recommend_books(ISBN, cosine_sim):
    # Get the index of the books that matches the ISBN
    idx = indices[ISBN]

    # Get the pairwsie similarity scores of all books with that book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    

    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df2[['Book_Title','ISBN']].iloc[book_indices], sim_scores

In [160]:
def get_recommendations(userid, cosine_sim):
    #userid = str(userid)
    ISBN = df_rating['ISBN'].loc[df_rating[df_rating['User-ID'] == userid]['Book-Rating'].idxmax()]

    recommendation, sim_score = recommend_books(ISBN, cosine_sim)

    df_recommend = pd.DataFrame(recommendation)
    df_recommend['Cosine_Similarity'] = sim_score
    df_recommend['Cosine_Similarity'] = df_recommend['Cosine_Similarity'].apply(lambda x: x[1])

    return df_recommend

In [161]:
##check which books read by User-ID 8
pd.concat([book_merged[book_merged['ISBN'].isin(list(df_rating[df_rating['User-ID'] == 8]['ISBN']))].reset_index(drop=True),df_rating[df_rating['User-ID'] == 8]['Book-Rating'].reset_index(drop=True)], axis = 1)

,Unnamed: 0.1,Unnamed: 0,Book_Title,language,page_count,categories,description,category,Features,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,1493,1493,A Second Chicken Soup for the Woman's Soul (Ch...,en,384.0,['Self-Help'],latest collection story celebrates shared expe...,selfhelp,selfhelp latest collection story celebrates sh...,1558746218,Jack Canfield,1998,Health Communications,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,http://images.amazon.com/images/P/1558746218.0...,0
1,1735,1735,The Kitchen God's Wife,en,420.0,['Fiction'],remarkable mesmerizing compelling entire world...,fiction,fiction remarkable mesmerizing compelling enti...,0399135782,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,0
2,6050,6050,Flu: The Story of the Great Influenza Pandemic...,en,330.0,['Social Science'],fascinating true story world deadliest disease...,social science,social science fascinating true story world de...,0374157065,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,0


In [162]:
df_recommend = get_recommendations(8, cosine_sim2)
df_recommend

,Book_Title,ISBN,Cosine_Similarity
12698,Flu : The Story Of The Great Influenza Pandemic,0743203984,1.000000
5704,Outbreak,042510687X,0.192506
5705,Outbreak,0399131876,0.192506
8667,Fatherland,0061006629,0.180989
8668,Fatherland,2266071173,0.180989
8669,Fatherland,0061008818,0.180989
8670,Fatherland,0679412735,0.180989
1910,The Illustrated Man (Grand Master Editions),055327449X,0.178458
1911,The Illustrated Man (Grand Master Editions),055327449x,0.178458
10301,Farewell to Manzanar: A True Story of Japanese...,0553272586,0.168443


In [163]:
# Define a function to get the top K similar books for each book
def get_similar_books(ISBN, cosine_sim, k):

    # Get the row index of the book in the cosine similarity matrix
    idx = book_merged[book_merged['ISBN'] == ISBN].index[0]

    # Get the pairwise cosine similarity scores for the book
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the book indices of the top K similar books
    book_indices = [i[0] for i in sim_scores[1:k+1]]

    # Return the top K similar books
    return df2[['Book_Title','ISBN']].iloc[book_indices]

In [164]:
df_recommend = get_similar_books("0743203984", cosine_sim2,5)
df_recommend

,Book_Title,ISBN
12698,Flu : The Story Of The Great Influenza Pandemic,0743203984
5704,Outbreak,042510687X
5705,Outbreak,0399131876
8667,Fatherland,0061006629
8668,Fatherland,2266071173
